# Dataset 만들기

## 1. 멜론_한국대중음악 수집

### 1) 모듈 불러오기

In [12]:
# 1. 크롬 드라이버 셀레니움 환경 설정하기
## 크롬드라이버 설치 -> 크롬 정보 확인하여 버전에 맞는걸로 Download
### https://googlechromelabs.github.io/chrome-for-testing/#stable

# 2. 셀레니움 설치 완료 후 필요한 모듈 불러오기
from selenium import webdriver                                          # webdriver : 크롬 가상 드라이버 실행
from selenium.webdriver.common.by import By                             # By : 실제 페이지 내용을 긁어올 때 수단을 설정하는 모듈
from selenium.webdriver.common.keys import Keys                         # Keys : 대신 키보드를 눌러주는 동작을 전달
from selenium.webdriver.support.ui import WebDriverWait                 # WebDriverWait : 대기가 필요한 상황에서 사용하기
from selenium.webdriver.support import expected_conditions as EC        # expected_conditions : 요소의 상태를 알아볼 때 사용
from selenium.webdriver.chrome.options import Options                   # Options : 크롬 드라이버의 옵션을 정함. ex) 크롬창 크기 등
from selenium.webdriver.chrome.service import Service                   # Service : Selenium에서 ChromeDriver의 경로를 설정하고 관리하는 역할
from selenium.webdriver.common.action_chains import ActionChains        # ActionCahins : Selenium에서 입력 장치를 사용하여 복잡한 사용자 동작을 자동화
from selenium.common.exceptions import NoSuchElementException           # NoSuchElementException : 지정한 조건에 맞는 요소를 찾을 수 없을 때 발생하는 예외 함수

# 3. BeautifulSoup 활용 할 모듈 불러오기
# from bs4 import BeautifulSoup                       
import time                                     # 시간 관련 작업을 간편하게 처리하기 위해 사용
# import math                                     # 수학적 계산을 위한 다양한 함수들을 제공
import pandas as pd 
# import pickle                                   # 데이터를 저장할 때 사용

from tqdm import tqdm                           # 진행률 보여주는 라이브러리 

### 2) Selenium으로 음원 chart 자료 크롤링

In [13]:
# WebDriver 설정 (ChromeDriver 경로 지정)
# service = Service(r"G:\내 드라이브\[Edu] wanted_poptenup\chromedriver-win64\chromedriver.exe")  # ChromeDriver 경로
service = Service(r"C:\Two_Kim_and_One_Lee\chromedriver-win64\chromedriver.exe")  # ChromeDriver 경로 (지민)
options = webdriver.ChromeOptions()

# 백그라운드 실행
# options.add_argument('--headless')                      # 브라우저를 GUI 없이 실행
# options.add_argument('--disable-gpu')                   # GPU를 비활성화
# options.add_argument('--no-sandbox')                    # Chcome의 샌드박스 보안 기능을 비활성화 (일부 제한적인 환경에서 안정적으로 실행되도록 설정)

# 브라우저 열기
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 멜론뮤직 홈페이지로 이동
url = "https://www.melon.com/"
driver.maximize_window()
# maximize_window()	창을 최대화합니다.
# minimize_window()	창을 최소화합니다.
# fullscreen_window()	창을 전체 화면 모드로 설정합니다.
# set_window_size(w, h)	창의 크기를 설정합니다.
# get_window_size()	창의 크기를 가져옵니다.
# set_window_position(x, y)	창의 위치를 설정합니다.
# get_window_position()	창의 위치를 가져옵니다.
# switch_to.window()	특정 창으로 전환합니다.
driver.get(url)                                             # 멜론 뮤직 차트로 진입
time.sleep(2)                                               # 페이지 로드 대기

# 멜론뮤직 '장르음악' 선택
melon_ganre = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[3]/a/span[2]')))
melon_ganre.click()

# 장르 선택하여 클릭
# second_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'')))
# second_box.click()

# '인기순' 클릭
popular_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="frm"]/div/div/div/a[2]')))
popular_box.click()

In [14]:
def collect_lyrics_in_list () : 
    songs = []
    columns = ['title', 'artist', 'lyrics', 'likes']
    song_data = pd.DataFrame(columns=columns)

    # 멜론 차트 list 만들기
    title = []
    artist = []
    lyrics = []
    likes = []

    # 기본 설정 값
    table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
    list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

    # 수집 루틴
    # tqdm 라이브러리로 진행 상황 바 표시
    for idx, meta in tqdm(enumerate(list_all), total=len(list_all), desc="Processing songs"):
        time.sleep(1)
        try :
            # 스크롤 동적으로 이동 : 인덱스 기반 스크롤
            driver.execute_script(f"window.scrollTo(0, {idx * 85});")

            # 특정 요소가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody>tr")))
            
            # 위치 찾아서 진입
            table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
            list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

            # 현재 곡 클릭
            list_click = list_all[idx].find_elements(By.CSS_SELECTOR, "td")
            list_click[3].click()
            song_click = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({int(idx+1)}) > td:nth-child(4) > div > a:not(.section_hitsong)")))
            song_click.click()

            # 제목 추가
            title_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".song_name"))).text
            title.append(title_text)

            # 아티스트 이름 추가
            artist_name = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".artist_name >span"))).text
            # //*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]
            # artist_row = artist_info.find_element(By.CSS_SELECTOR, "tbody>tr>td")
            # artist_name = artist_row.find_element(By.CSS_SELECTOR, "a").text
            artist.append(artist_name)

            lyrics_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".lyric")))
            if lyrics_element:
                # 요소가 존재하는 경우
                lyrics_text = lyrics_element.text  # 첫 번째 요소 선택
            else:
                # 요소가 존재하지 않는 경우
                lyrics_text = ""
                print(lyrics_element)
          
            # 요소가 존재하는 경우에만 텍스트를 가져옴
            # lyrics_text = lyrics_element.text if lyrics_element else ""
            # lyrics.append(lyrics_text)

            # 좋아요 개수 추가
            likes_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".cnt"))).text
            likes.append(likes_text)

            # 뒤로가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="songList"]')))
            
        except :
            print(f"오류 발생", {idx})        
            break
        
        # 곡 정보 저장
        song_data = pd.DataFrame(
            {
                "title": title,
                "artist": artist,
                "lyrics": lyrics,
                "likes": likes
            }
        )

    return song_data

In [15]:
collect_lyrics_in_list()

Processing songs:   0%|          | 0/50 [00:00<?, ?it/s]

Processing songs:   0%|          | 0/50 [00:02<?, ?it/s]


ValueError: All arrays must be of the same length

In [4]:
# next 버튼 누를지 보는 함수 
def is_in_sequence(num):
    # num이 11 이상인지 확인
    if num < 11:
        return False
    
    # num이 11에서 시작하는 수열인지 확인
    return (num - 11) % 10 == 0

In [5]:
#Xpath : //*[@id="pageObjNavgation"]/div/span/a[1]
#Xpath : //*[@id="pageObjNavgation"]/div/a

# 1. 곡 수집 -> 함수로 실행 
# 2. 50곡 완료 시 next page이동 (10n번째 -> next버튼)
columns = ['title', 'artist', 'lyrics', 'likes']
merge_songs = pd.DataFrame(columns=columns)

idx = 0
while True:
    # 다음 페이지로 이동
    try:
        idx += 1 
        print("="*25,idx,"페이지 데이터 곡 수집","="*25)
        # 곡 정보 가져오기
        curr_songs = collect_lyrics_in_list()
        
        merge_songs = pd.concat([merge_songs, curr_songs], ignore_index = True)

        if is_in_sequence(idx): # 
            # 다음 페이지로 이동 (NEXT 버튼 클릭)
            print(f">>>>>>>>>>>>>>>>{idx} 페이지로 다음 버튼 클릭 합니다>>>>>>>>>>>>>>>>")
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.next'))
            )
            next_button.click()
        else:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{idx}]'))  # 다음 페이지 버튼의 XPath
            )
            next_button.click()
        
    except Exception as e:
        print("다음 페이지가 없습니다:", e)
        break  # 더 이상 페이지가 없으면 반복 종료



========================= 1 페이지 데이터 곡 수집 =========================


Processing songs: 100%|██████████| 50/50 [01:37<00:00,  1.94s/it]


========================= 2 페이지 데이터 곡 수집 =========================


Processing songs: 100%|██████████| 50/50 [01:37<00:00,  1.95s/it]


========================= 3 페이지 데이터 곡 수집 =========================


...

Processing songs: 100%|██████████| 50/50 [01:35<00:00,  1.90s/it]


========================= 6 페이지 데이터 곡 수집 =========================


Processing songs:  26%|██▌       | 13/50 [00:36<01:45,  2.84s/it]

오류 발생 {13}


다음 페이지가 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE2F80D5+2992373]
	(No symbol) [0x00007FF7ADF8BFD0]
	(No symbol) [0x00007FF7ADE2590A]
	(No symbol) [0x00007FF7ADE7926E]
	(No symbol) [0x00007FF7ADE7955C]
	(No symbol) [0x00007FF7ADEC27D7]
	(No symbol) [0x00007FF7ADE9F3AF]
	(No symbol) [0x00007FF7ADEBF584]
	(No symbol) [0x00007FF7ADE9F113]
	(No symbol) [0x00007FF7ADE6A918]
	(No symbol) [0x00007FF7ADE6BA81]
	GetHandleVerifier [0x00007FF7AE356A2D+3379789]
	GetHandleVerifier [0x00007FF7AE36C32D+3468109]
	GetHandleVerifier [0x00007FF7AE360043+3418211]
	GetHandleVerifier [0x00007FF7AE0EC78B+847787]
	(No symbol) [0x00007FF7ADF9757F]
	(No symbol) [0x00007FF7ADF92FC4]
	(No symbol) [0x00007FF7ADF9315D]
	(No symbol) [0x00007FF7ADF82979]
	BaseThreadInitThunk [0x00007FFE6775259D+29]
	RtlUserThreadStart [0x00007FFE6996AF38+40]



In [7]:
merge_songs

,title,artist,lyrics,likes
0,나는 반딧불,<selenium.webdriver.remote.webelement.WebEleme...,나는 내가 빛나는 별인 줄 알았어요\n한 번도 의심한 적 없었죠\n몰랐어요 난 내가...,"84,895"
1,내게 사랑이 뭐냐고 물어본다면,<selenium.webdriver.remote.webelement.WebEleme...,뜨겁게 사랑했던\n계절을 지나\n처음과는 조금은 달라진\n우리 모습을\n걱정 하진 ...,"49,445"
2,소나기,<selenium.webdriver.remote.webelement.WebEleme...,그치지 않기를 바랬죠\n처음 그대 내게로 오던 그날에\n잠시 동안 적시는\n그런 비...,"169,458"
3,천상연,<selenium.webdriver.remote.webelement.WebEleme...,아니길 바랬었어\n꿈이길 기도했지\n너 없는 가슴으로 살아가야 하는 건\n내게는 너...,"114,764"
4,슬픈 초대장,<selenium.webdriver.remote.webelement.WebEleme...,내 야윈 손위로 온 초대장 위에\n널 데려간다는 그와 네 이름\n오래전 헤어지던 날...,"61,072"
...,...,...,...,...
258,위로,<selenium.webdriver.remote.webelement.WebEleme...,세상과 다른 눈으로 나를 사랑하는\n세상과 다른 맘으로 나를 사랑하는\n그런 그대가...,"99,229"
259,끝,<selenium.webdriver.remote.webelement.WebEleme...,한참을 아무 말도 없었다\n어색한 공기만 우릴 감싸고 있다\n너는 고개를 돌린 채로...,"166,015"
260,내 생에 아름다운,<selenium.webdriver.remote.webelement.WebEleme...,Don't walk away I'm falling down\nDon't walk a...,"175,419"
261,너는 어땠을까,<selenium.webdriver.remote.webelement.WebEleme...,언제부터인지 습관처럼 연락하고\n마주한 시간이 더는 설레질 않아\n하루 종일 반복되...,"116,794"


In [ ]:
import datetime

#데이터 프레임 저장
now = datetime.datetime.now()
address = '../01_data_모음/'

# os.chdir("./data")
# song_data.to_csv(address, index=False, encoding='utf-8-sig')
merge_songs.to_csv(path_or_buf=address+'melon.csv')
# song_data.to_csv('melon{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

### 3) 곡정보 담기

In [ ]:
# 멜론 차트 list 만들기
title = []
artist = []
lyrics = []
likes = []

# 기본 설정 값
table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

# 수집 루틴
for idx in range(0, len(list_all)):
    time.sleep(1)
    try :
        # 스크롤 동적으로 이동 : 인덱스 기반 스크롤
        driver.execute_script(f"window.scrollTo(0, {idx * 85});")

        # 특정 요소가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody>tr")))
        
        # 위치 찾아서 진입
        table_all = driver.find_element(By.XPATH, '//*[@id="songList"]')
        list_all = driver.find_elements(By.CSS_SELECTOR, "tbody>tr")

        # 현재 곡 클릭
        list_click = list_all[idx].find_elements(By.CSS_SELECTOR, "td")
        list_click[3].click()
        song_click = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({int(idx+1)}) > td:nth-child(4) > div > a:not(.section_hitsong)")))
        song_click.click()

        # 제목 추가
        title_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".song_name"))).text
        title.append(title_text)

        # 아티스트 이름 추가
        artist_name = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".artist_name")))
        # artist_row = artist_info.find_element(By.CSS_SELECTOR, "tbody>tr>td")
        # artist_name = artist_row.find_element(By.CSS_SELECTOR, "a").text
        artist.append(artist_name)

        # 가사 텍스트 추가
        lyrics_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".lyric"))).text
        lyrics.append(lyrics_text)

        # 좋아요 개수 추가
        likes_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".cnt"))).text
        likes.append(likes_text)
        
        # 중간 확인
        # if list(range(1, 5000+1, 100) in len(title) :
        #         print(len(title))
        # else :
        #     continue

        # 뒤로가기
        driver.back()
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="songList"]')))
    except :
        print(f"오류 발생", {idx})        
        break

### 4) 수집량 확인

In [ ]:
print(len(title))
print(len(artist))
print(len(lyrics))
print(len(likes))

## 2. 파일 내보내기

### 1) 수집 데이터 프레임화

In [40]:
# 데이터 취합하기
data = pd.DataFrame(
    {
        "제목" : title,
        "가수" : artist,
        "가사" : lyrics,
        "좋아요" : likes
    }
)

In [ ]:
# 데이터 확인하기
data.head(10)

### 2) 데이터 내보내기

In [43]:
# 데이터 csv파일로 내보내기
data.to_csv(r"G:\내 드라이브\[Edu] wanted_poptenup\01_프로젝트 모음\25.01_프로젝트#2_'BE MY MUSE' 작사 공모전\(Data)\melon_한국대중음악_발라드(TOP50).csv", index=True, encoding='utf-8')